# Workshop 5: Gradient Descent
## Data Science for Mathematicians

This workshop implements the gradient descent family from first principles, using the **Boston Housing dataset** to predict median home values. We move from the abstract calculus of loss landscapes to scalable stochastic algorithms.

**Key concepts covered:**
- Computing and visualizing the OLS loss landscape
- Gradient and Hessian of the MSE loss (analytical vs. numerical)
- Batch gradient descent: derivation and implementation from scratch
- The learning rate: convergence, divergence, and the stability bound
- Convexity: verifying the OLS loss is convex via the Hessian
- Stochastic gradient descent (SGD): unbiasedness and per-step variance
- Mini-batch gradient descent: variance reduction and the $1/b$ scaling law
- Comparison of all three variants; Scikit-learn `SGDRegressor`

## Section 0: Setup and Data Loading

***

## Section 1: The Loss Landscape

**Theory recap.**  For OLS with parameter vector $\boldsymbol{\beta} \in \mathbb{R}^p$, the mean squared error (MSE) loss is
$$
L(\boldsymbol{\beta}) = \frac{1}{n}\|\mathbf{X}\boldsymbol{\beta} - \mathbf{y}\|_2^2.
$$
This defines a scalar field over parameter space---the **loss landscape**.  For $p = 1$ (plus an intercept) it is a paraboloid in $\mathbb{R}^3$; for $p = 1$ with the intercept fixed it collapses to a parabola in $\mathbb{R}^2$, which we can plot directly.

### Question 1.1: Plot the 1D Loss Landscape

Fix $\beta_0 = 0$ and sweep $\beta_1$ over $[-2, 3]$.  For each value compute the MSE loss
$$L(\beta_1) = \frac{1}{n}\sum_{i=1}^n (\beta_1 x_i - y_i)^2.$$
Plot the loss curve, mark the closed-form minimum, and verify that the landscape is a convex parabola.

**Interpretation.** The loss landscape is a convex parabola---a single bowl with one minimum and no local minima. This guarantees that gradient descent will find the global optimum regardless of initialization.

### Question 1.2: Analytical vs. Numerical Gradient

The gradient of the OLS loss is
$$\nabla L(\boldsymbol{\beta}) = \frac{2}{n}\mathbf{X}^T(\mathbf{X}\boldsymbol{\beta} - \mathbf{y}).$$

Implement both the loss function and its gradient.  Verify the analytical formula against a finite-difference (numerical) approximation at a test point $\boldsymbol{\beta} = [0,\, 0.5]^T$.  The maximum absolute error should be below $10^{-5}$.

***

## Section 2: Batch Gradient Descent

**Theory recap.** Derived from the first-order Taylor expansion, the gradient descent update rule is
$$
\boldsymbol{\beta}_{k+1} = \boldsymbol{\beta}_k - \eta\,\nabla L(\boldsymbol{\beta}_k),
$$
where $\eta > 0$ is the **learning rate**.  Batch GD uses all $n$ samples to compute the exact gradient at each step, guaranteeing monotonic loss decrease (for sufficiently small $\eta$).

### Question 2.1: Implement Batch Gradient Descent from Scratch

Write a function `batch_gd(X, y, eta, n_iters)` that:
1. Initializes $\boldsymbol{\beta} = \mathbf{0}$
2. At each iteration computes the full gradient using all $n$ samples
3. Updates $\boldsymbol{\beta}$ and records the loss
4. Stops early if $\|\nabla L\|_2 < \texttt{tol}$

Run it with $\eta = 0.05$ for up to 500 iterations.  Compare the result with the closed-form OLS solution.

### Question 2.2: Plot the Convergence Curve

Plot the loss history on a **log scale** to reveal the exponential convergence rate characteristic of gradient descent on a strongly convex quadratic.  Mark the optimal loss $L^* = L(\boldsymbol{\beta}^*)$ as a horizontal dashed line.

**Interpretation.** The log-scale plot reveals a straight-line decay, confirming **geometric (exponential) convergence**. This is a consequence of the OLS loss being a strongly convex quadratic: each iteration reduces the loss by a constant factor determined by the learning rate and the Hessian eigenvalues.

***

## Section 3: The Learning Rate

**Theory recap.**  For OLS, the Hessian is $\mathbf{H}_L = \frac{2}{n}\mathbf{X}^T\mathbf{X}$ (constant).  Batch GD converges if and only if
$$
\eta < \frac{2}{\lambda_{\max}(\mathbf{H}_L)},
$$
where $\lambda_{\max}$ is the largest eigenvalue of $\mathbf{H}_L$.  Violating this bound causes geometric divergence.

### Question 3.1: Find the Theoretical Stability Bound

Compute $\mathbf{H}_L = \frac{2}{n}\mathbf{X}^T\mathbf{X}$, find its eigenvalues, and derive the maximum stable learning rate $\eta_{\max} = 2/\lambda_{\max}$.  Then run batch GD with four learning rates: one far below $\eta_{\max}$, one near $\eta_{\max}$, one just above, and one well above.  Visualize all four convergence curves on the same plot.

**Interpretation.** Any $\eta > \eta_{\max}$ causes the iterates to diverge geometrically---the step overshoots the minimum and the loss grows at each iteration. Below the bound, larger $\eta$ yields faster convergence (up to the limit), while $\eta$ too small wastes many iterations taking negligibly small steps.

***

## Section 4: Convexity Analysis

**Theory recap.** A twice-differentiable function $f$ is convex if and only if its Hessian is positive semidefinite (PSD) everywhere: $\mathbf{H}_f(\mathbf{x}) \succeq \mathbf{0}$ for all $\mathbf{x}$.  The global optimality theorem then guarantees that any local minimum is a global minimum.

For OLS: $\mathbf{H}_L = \frac{2}{n}\mathbf{X}^T\mathbf{X}$.  Since $\mathbf{z}^T\mathbf{H}_L\mathbf{z} = \frac{2}{n}\|\mathbf{X}\mathbf{z}\|_2^2 \ge 0$ for all $\mathbf{z}$, the loss is always convex.

### Question 4.1: Verify Convexity via the Hessian Spectrum

For both the 2-parameter dataset (intercept + RM) and the full 14-parameter dataset (all features), compute the Hessian $\mathbf{H}_L = \frac{2}{n}\mathbf{X}^T\mathbf{X}$ and verify that all eigenvalues are non-negative.  Also verify the PSD condition directly by checking $\mathbf{z}^T\mathbf{H}_L\mathbf{z} \ge 0$ for 1000 random vectors $\mathbf{z}$.

### Question 4.2: Visualize the Loss Landscape's Curvature

Plot the eigenvalue spectrum of the full Hessian as a bar chart.  The ratio $\lambda_{\max}/\lambda_{\min}$ is the **condition number**---a large condition number means the landscape is "elongated" (ill-conditioned), making gradient descent slow.  Report the condition number.

**Interpretation.** All eigenvalues are strictly positive, confirming the Hessian is **positive definite** (PD), which means the loss is **strictly convex** with a unique global minimum.  A large condition number reveals an elongated loss landscape: gradient descent makes fast progress along high-curvature directions but creeps along low-curvature ones---this motivates adaptive learning rate methods like Adam.

***

## Section 5: Stochastic Gradient Descent (SGD)

**Theory recap.**  SGD replaces the full-batch gradient with a single-sample approximation:
$$
\boldsymbol{\beta}_{k+1} = \boldsymbol{\beta}_k - \eta_k\,\nabla L_{i_k}(\boldsymbol{\beta}_k),
\qquad i_k \sim \text{Uniform}\{1,\dots,n\}.
$$
By the **unbiasedness lemma**, $\mathbb{E}_{i}[\nabla L_i(\boldsymbol{\beta})] = \nabla L(\boldsymbol{\beta})$, so each step is noisy but statistically correct.  One epoch contains $n$ update steps.

### Question 5.1: Implement SGD from Scratch

Write `sgd(X, y, eta, n_epochs)` that shuffles data each epoch and applies one update per sample.  Track:
- **Step losses**: MSE after every single update (noisy trajectory)
- **Epoch losses**: MSE at the end of each epoch (smoother signal)

Verify the unbiasedness lemma empirically: average 1000 single-sample gradients at a fixed $\boldsymbol{\beta}$ and compare with the full-batch gradient.

### Question 5.2: Compare SGD and Batch GD

Run SGD for 20 epochs and batch GD for 20 iterations.  Plot:
- Left: SGD step-level loss (all $20n$ steps) showing the noisy sawtooth pattern
- Right: Per-epoch loss comparison between SGD and batch GD

**Interpretation.** SGD's step-level loss shows the characteristic **sawtooth pattern**: rapid descent within an epoch interrupted by occasional upward spikes when a high-variance sample pulls the parameters the wrong way.  Per epoch, SGD can match or exceed batch GD's progress by performing $n$ updates instead of one---but its final accuracy is limited by gradient variance under a fixed learning rate.

***

## Section 6: Mini-Batch Gradient Descent

**Theory recap.**  Mini-batch GD uses a random batch of $b$ samples per update:
$$
\mathbf{g}_k = \frac{1}{b}\sum_{i \in \mathcal{B}_k} \nabla L_i(\boldsymbol{\beta}_k),
\qquad \boldsymbol{\beta}_{k+1} = \boldsymbol{\beta}_k - \eta\,\mathbf{g}_k.
$$
The gradient estimator is still unbiased, but its variance shrinks by $1/b$:
$$\operatorname{Var}(\mathbf{g}_k) = \frac{1}{b}\operatorname{Var}(\nabla L_i(\boldsymbol{\beta})).$$

### Question 6.1: Implement Mini-Batch GD and Explore Batch Size

Write `minibatch_gd(X, y, eta, batch_size, n_epochs)`.  Run it with batch sizes $b \in \{1, 16, 64, n\}$ (corresponding to SGD, small batch, medium batch, and full batch) and plot the per-epoch convergence curves on the same axes.

### Question 6.2: Verify the $1/b$ Variance Scaling Law

The theory predicts $\operatorname{Var}(\mathbf{g}_k) = \frac{1}{b}\operatorname{Var}(\nabla L_i)$.  Empirically estimate the gradient variance at $\boldsymbol{\beta} = \boldsymbol{\beta}^*$ for a range of batch sizes $b \in \{1, 2, 4, \dots, 256\}$ by averaging over 500 random mini-batches.  Plot gradient variance vs. $b$ on a log-log scale and verify the $-1$ slope.

**Interpretation.** The empirical slope of $\approx -1$ on the log-log plot confirms the $1/b$ scaling law: **doubling the batch size halves the gradient variance**.  This is the fundamental trade-off of mini-batch GD---each doubling of $b$ costs twice as many FLOPs per update, but in practice modern GPU parallelism makes a mini-batch of 64 almost as fast as a mini-batch of 1.

***

## Section 7: Full-Feature Regression and Scikit-learn

We now apply all three gradient descent variants to the **full 13-feature Boston Housing dataset** and compare against Scikit-learn's optimized `SGDRegressor`.

### Question 7.1: Train All Three Variants on the Full Dataset

Using all 13 features (standardized), run batch GD, SGD, and mini-batch GD for 50 epochs each.  Plot their per-epoch loss curves on the same axes alongside the closed-form OLS optimum.

### Question 7.2: Scikit-learn SGDRegressor and Feature Importance

Use `sklearn.linear_model.SGDRegressor` to fit the same data.  Then extract the OLS coefficients (from the closed-form solution) and visualize which features most strongly influence the predicted house price.

**Interpretation.**  
- **LSTAT** (% lower-status population) has the largest negative coefficient: neighborhoods with more lower-status residents have significantly lower median house prices.
- **RM** (average rooms) has the largest positive coefficient: more rooms correlates with higher prices.
- **PTRATIO** (pupil-teacher ratio) is also strongly negative: better schools (lower ratio) predict higher prices.

These signs align with domain knowledge, providing a sanity check that the gradient descent algorithm converged to the correct solution.

***

## Summary

| Method | Samples/update | Updates/epoch | Cost/update | Convergence path |
|--------|---------------|---------------|-------------|------------------|
| Batch GD | $n$ (all) | 1 | $O(np)$ | Smooth, monotone |
| SGD | 1 | $n$ | $O(p)$ | Noisy, fast early |
| Mini-Batch GD | $b$ | $n/b$ | $O(bp)$ | Smooth, fast |

**Key results from this workshop:**

1. The OLS loss landscape is a convex paraboloid: the Hessian $\mathbf{H}_L = \frac{2}{n}\mathbf{X}^T\mathbf{X}$ is PSD everywhere, guaranteeing a unique global minimum.
2. Batch GD converges geometrically (linear decay on log scale) with rate determined by $\eta \cdot \lambda_{\max}$. The stability bound $\eta < 2/\lambda_{\max}$ is sharp.
3. The stochastic gradient $\nabla L_i$ is an unbiased estimator of $\nabla L$; averaging over $b$ samples reduces variance by $1/b$ (the $1/b$ scaling law).
4. Mini-batch GD with $b \in \{32, 64\}$ is the practical standard: it matches SGD's update frequency while exploiting GPU parallelism and reducing gradient variance.